# TP3-4 IASD Lancelin Foussadier et Romain Alves

In [58]:
from sklearn.neural_network import MLPClassifier
from random import randint
mlp_clf_AND = MLPClassifier(hidden_layer_sizes=(),
                        max_iter = 300,activation = 'identity',
                        solver = 'lbfgs')
X = [ [0., 0.], [0., 1.], [1., 0.], [1., 1.]] # Entrées
y = [0, 0, 0, 1] # Résultats attendus
mlp_clf_AND.fit(X,y)
mlp_clf_AND.predict(X)
X_test = []
for i in range(60):
    x = (randint(0,1),randint(0,1))
    X_test.append(x)
mlp_clf_AND.predict(X_test)
mlp_clf_OR = MLPClassifier(hidden_layer_sizes=(),
                        max_iter = 300,activation = 'identity',
                        solver = 'lbfgs')
y_OR = [0,1,1,1]
mlp_clf_OR.fit(X,y_OR)
mlp_clf_OR.predict(X_test)

mlp_clf_XOR_0 = MLPClassifier(hidden_layer_sizes=(),
                        max_iter = 300,activation = 'identity',
                        solver = 'lbfgs')
y_XOR = [0,1,1,0]
mlp_clf_XOR_0.fit(X,y_XOR)
mlp_clf_XOR_0.predict(X_test)

#On observe que le Classifieur a eu une prédiction "aléatoire" tout a été prédis à 0. 
#Cela peut être due au fait que les jeux de tests n'ont pas assez entrainés le modèle
mlp_clf_XOR_1 = MLPClassifier(hidden_layer_sizes=(4,2),
                        max_iter = 300,activation = 'identity',
                        solver = 'lbfgs')


### On modifie ici le pour tester AND, OR et XOR
mlp_clf_XOR_1.fit(X,y_XOR)
y_test = mlp_clf_XOR_1.predict(X_test)

x_model =[]
for i in X_test:
    if (i[0] == 1) and (i[1] == 1):
        x_model.append(0)
    elif  i[0] == 0 and i[1] == 0:
        x_model.append(0)
    else :
        x_model.append(1)


In [59]:
from sklearn.metrics import accuracy_score
print(accuracy_score(x_model,y_test))
print(x_model)
print(y_test)

0.5
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]
[0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 0 1 0 0 1
 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0]


# On remarque que le calssifier prédis la bonne valeure entre 50 et 80% du temps pour chaque opération logique.

In [60]:
mlp_clf_XOR_2 = MLPClassifier(hidden_layer_sizes=(4,2),activation = 'tanh',
                        solver = 'lbfgs')
mlp_clf_XOR_2.fit(X,y_XOR)
y_test = mlp_clf_XOR_2.predict(X_test)
accuracy_score(x_model,y_test)
#Les résultats sont bons dans 100% des cas même après plusieurs apprentissages

1.0

# Sans neurones les résultats sont très mauvais, ils sont aléatoires entre 80% et 20%. Avec 4 neurones par couche et deux couches les resultats s'améliorent peu. Et avec tanh les résultats sont très bons. 

In [61]:
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split
dataset = load_digits()
X = dataset.data # I n p u t s
y = dataset.target # A s s o c i a t e d o u t p u t s
train_X , test_X , train_y , test_y = train_test_split (X, y, test_size =0.1)

mlp = MLPClassifier(hidden_layer_sizes=(100),max_iter = 1000, activation = 'identity', solver = 'lbfgs')
mlp.fit(train_X,train_y)
y_model = mlp.predict(test_X)
accuracy_score(y_model,test_y)

0.9611111111111111

# Après plusieurs tests, on peut estimer que les meilleurs paramètres sont 'identity', 'lbfgs' et 100 couches de neurones.

In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1309,), (0.3084,)) # Moyenne et ecart-type calculées auparavant
])
trainset = datasets.MNIST(root='./data', train=True,download=True, transform=transform) # On crée un dataset
trainloader = DataLoader(trainset, batch_size=64,shuffle=True) # On crée un dataloader
testset = datasets.MNIST(root='./data', train=False,download=True, transform=transform) # On crée un dataset
testloader = DataLoader(testset, batch_size=64,shuffle=False) # On crée un dataloader
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader): # On itère sur les batchs
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad() # On met les gradients à 0
        # forward + backward + optimize
        outputs = net(inputs) # On calcule les sorties du réseau
        loss = criterion(outputs, labels) # On calcule la loss
        loss.backward() # On calcule les gradients
        optimizer.step() # On met à jour les poids
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99: # On affiche la loss tous les 100 batchs
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            running_loss = 0.0

# functions to show an image
def imshow(img):
    img = img / 0.3084 + 0.1309     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Calcule de la précision (accuracy) sur le jeu de test
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# Accuracy par classe
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[1, 100] loss: 2.289447727203369
[1, 200] loss: 2.247607374191284
[1, 300] loss: 2.0605725729465485
[1, 400] loss: 1.1821289372444153
[1, 500] loss: 0.5977187219262123
[1, 600] loss: 0.4634158892929554
[1, 700] loss: 0.39198525846004484
[1, 800] loss: 0.39240347757935523
[1, 900] loss: 0.36521195456385613
[2, 100] loss: 0.31423467613756656
[2, 200] loss: 0.2762450975179672
[2, 300] loss: 0.2678286175429821
[2, 400] loss: 0.26454481422901155
[2, 500] loss: 0.2355415190011263
[2, 600] loss: 0.2312073064595461
[2, 700] loss: 0.22192187286913395
[2, 800] loss: 0.19909939561039208
[2, 900] loss: 0.1872195026278496
Accuracy of the network on the 10000 test images: 94 %
Accuracy for class:

# On remarque que la précision est entre 95 et 98%  ce qui est très bon comparé au classifier qui lui oscillait entre 85 et 96%.
Nous allons maintenant tester de faire varié les paramètres du réseau de neurones pour voir si on peut améliorer la précision comme on l'avait fait pour le classifier.
En commenceant par la taille des noyaux de convolution. Que l'on va faire varier à 3.

In [63]:
import torch
import torch.nn as nn


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net2 = Net()
print(net2)

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1309,), (0.3084,)) # Moyenne et ecart-type calculées auparavant
])
trainset = datasets.MNIST(root='./data', train=True,download=True, transform=transform) # On crée un dataset
trainloader = DataLoader(trainset, batch_size=64,shuffle=True) # On crée un dataloader
testset = datasets.MNIST(root='./data', train=False,download=True, transform=transform) # On crée un dataset
testloader = DataLoader(testset, batch_size=64,shuffle=False) # On crée un dataloader
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader): # On itère sur les batchs
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad() # On met les gradients à 0
        # forward + backward + optimize
        outputs = net2(inputs) # On calcule les sorties du réseau
        loss = criterion(outputs, labels) # On calcule la loss
        loss.backward() # On calcule les gradients
        optimizer.step() # On met à jour les poids
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99: # On affiche la loss tous les 100 batchs
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            running_loss = 0.0

# functions to show an image
def imshow(img):
    img = img / 0.3084 + 0.1309     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Calcule de la précision (accuracy) sur le jeu de test
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net2(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# Accuracy par classe
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net2(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[1, 100] loss: 2.296995027065277
[1, 200] loss: 2.26899742603302
[1, 300] loss: 2.19838663816452
[1, 400] loss: 1.8708905899524688
[1, 500] loss: 1.0042804646492005
[1, 600] loss: 0.6021834731101989
[1, 700] loss: 0.4854024292528629
[1, 800] loss: 0.41560176491737366
[1, 900] loss: 0.363036797195673
[2, 100] loss: 0.30029556527733803
[2, 200] loss: 0.2763645215332508
[2, 300] loss: 0.2653337824344635
[2, 400] loss: 0.23153631981462242
[2, 500] loss: 0.2309646026790142
[2, 600] loss: 0.20986890681087972
[2, 700] loss: 0.20417704679071902
[2, 800] loss: 0.21192378524690866
[2, 900] loss: 0.16827741261571647
Accuracy of the network on the 10000 test images: 94 %
Accuracy for class: 0  

# On voit qu'avec des noyaux de taille moins grand le réseau de neurones est moins performant.
On va maintenant essayer de faire varier la taille des features maps.

In [64]:
import torch
import torch.nn as nn


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 25, 5)
        self.conv2 = nn.Conv2d(25, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net3 = Net()
print(net3)

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1309,), (0.3084,)) # Moyenne et ecart-type calculées auparavant
])
trainset = datasets.MNIST(root='./data', train=True,download=True, transform=transform) # On crée un dataset
trainloader = DataLoader(trainset, batch_size=64,shuffle=True) # On crée un dataloader
testset = datasets.MNIST(root='./data', train=False,download=True, transform=transform) # On crée un dataset
testloader = DataLoader(testset, batch_size=64,shuffle=False) # On crée un dataloader
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net3.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader): # On itère sur les batchs
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad() # On met les gradients à 0
        # forward + backward + optimize
        outputs = net3(inputs) # On calcule les sorties du réseau
        loss = criterion(outputs, labels) # On calcule la loss
        loss.backward() # On calcule les gradients
        optimizer.step() # On met à jour les poids
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99: # On affiche la loss tous les 100 batchs
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            running_loss = 0.0

# functions to show an image
def imshow(img):
    img = img / 0.3084 + 0.1309     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Calcule de la précision (accuracy) sur le jeu de test
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net3(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# Accuracy par classe
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net3(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Net(
  (conv1): Conv2d(1, 25, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(25, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[1, 100] loss: 2.294354889392853
[1, 200] loss: 2.2426527094841004
[1, 300] loss: 1.9542124843597413
[1, 400] loss: 0.9202047330141068
[1, 500] loss: 0.5156365664303303
[1, 600] loss: 0.4043690373003483
[1, 700] loss: 0.35698885187506674
[1, 800] loss: 0.2991925658285618
[1, 900] loss: 0.2642312593013048
[2, 100] loss: 0.23278339177370072
[2, 200] loss: 0.23320513352751732
[2, 300] loss: 0.20033549554646016
[2, 400] loss: 0.19469706673175097
[2, 500] loss: 0.1694080750271678
[2, 600] loss: 0.16031047824770212
[2, 700] loss: 0.1692545835673809
[2, 800] loss: 0.1524422680027783
[2, 900] loss: 0.1538460937514901
Accuracy of the network on the 10000 test images: 96 %
Accuracy for clas

# On voit qu'avec des features maps de taille plus petite le réseau de neurones est moins performant. Néanmoins si l'on augmente on a peu de variation de la précision.
On va maintenant essayer de faire varier le nombre de couches de neurones.

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net4 = Net()
print(net4)

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1309,), (0.3084,)) # Moyenne et ecart-type calculées auparavant
])
trainset = datasets.MNIST(root='./data', train=True,download=True, transform=transform) # On crée un dataset
trainloader = DataLoader(trainset, batch_size=64,shuffle=True) # On crée un dataloader
testset = datasets.MNIST(root='./data', train=False,download=True, transform=transform) # On crée un dataset
testloader = DataLoader(testset, batch_size=64,shuffle=False) # On crée un dataloader
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net4.parameters(), lr=0.010, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader): # On itère sur les batchs
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad() # On met les gradients à 0
        # forward + backward + optimize
        outputs = net4(inputs) # On calcule les sorties du réseau
        loss = criterion(outputs, labels) # On calcule la loss
        loss.backward() # On calcule les gradients
        optimizer.step() # On met à jour les poids
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99: # On affiche la loss tous les 100 batchs
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            running_loss = 0.0
    
import numpy as np
from matplotlib import pyplot as plt

# functions to show an image
def imshow(img):
    img = img / 0.3084 + 0.1309     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Calcule de la précision (accuracy) sur le jeu de test
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net4(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# Accuracy par classe
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net4(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[1, 100] loss: 1.9088922065496445
[1, 200] loss: 0.3745897134393454
[1, 300] loss: 0.17896830633282662
[1, 400] loss: 0.13307213144376873
[1, 500] loss: 0.12609489487484096
[1, 600] loss: 0.10703282654285431
[1, 700] loss: 0.09714987510815262
[1, 800] loss: 0.08491703028790652
[1, 900] loss: 0.08403366690501571
[2, 100] loss: 0.06491851876955479
[2, 200] loss: 0.06315013991901651
[2, 300] loss: 0.07199754202272743
[2, 400] loss: 0.07205491409637034
[2, 500] loss: 0.05694212828762829
[2, 600] loss: 0.05835183144547045
[2, 700] loss: 0.05182410222943872
[2, 800] loss: 0.05456122806528583
[2, 900] loss: 0.06427852617576718
Accuracy of the network on the 10000 test images: 98 %
Accuracy

# On voit qu'avec un learing rate de 0.01 le réseau de neurones est meilleur dans ses predictions de quelques pourcents comparé a un learing rate de 0.001.
Néanmoins, si l'on augmente à plus ont de moins en moins bons résultats. Il faut donc trouver un juste milieu.